<a href="https://colab.research.google.com/github/vs-152/FL-Contributions-Incentives-Project/blob/main/ISO_CIFAR10_LOO_FINAL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import numpy as np
!pip install pulp
import pulp
import copy
import time
from sklearn.model_selection import StratifiedShuffleSplit
import torchvision
from torchvision.datasets import CIFAR10
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader, TensorDataset
from torch.utils.data.sampler import SubsetRandomSampler
from itertools import chain, combinations
from tqdm import tqdm
from scipy.special import comb
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
def print_solution(model):
    """Prints solution of the model nicely!"""

    print(f"status: {model.status}, {pulp.LpStatus[model.status]}")
    print(f"objective: {model.objective.value()}")
    for var in model.variables():
        print(f"{var.name}: {round(var.value(),3)}")

def noisify_MNIST(noise_rate, noise_type, x, y, perm=[], **kwargs):
    '''Returns a symmetrically noisy dataset
    or a an asymmetrically noisy dataset with permutation matrix perm.
    '''
    if (noise_rate == 0.):
        return y, []
    if 'seed' in kwargs:
        _, noise_idx = next(
            iter(StratifiedShuffleSplit(
                n_splits=1,
                test_size=noise_rate,
                random_state=kwargs['seed']).split(x, y)))
    else:
        _, noise_idx = next(iter(StratifiedShuffleSplit(
            n_splits=1, test_size=noise_rate).split(x, y)))
    y_noisy = y.copy()
    if (noise_type == 'symmetric'):
        for i in noise_idx:
            t1 = np.arange(10)
            t2 = np.delete(t1, y[i])
            y_noisy[i] = np.random.choice(t2, 1)
    elif (noise_type == 'asymmetric'):
        pure_noise = perm[y]
        for i in noise_idx:
            if (perm[y[i]] == y[i]):
                noise_idx = np.delete(noise_idx, np.where(noise_idx == i))
            else:
                y_noisy[i] = pure_noise[i]

    return y_noisy, noise_idx

def mnist_iid(dataset, num_users):
    """
    Sample I.I.D. client data from MNIST dataset
    :param dataset:
    :param num_users:
    :return: dict of image index
    """
    num_items = int(len(dataset)/num_users)
    dict_users, all_idxs = {}, [i for i in range(len(dataset))]
    for i in range(num_users):
        dict_users[i] = set(np.random.choice(all_idxs, num_items,
                                             replace=False))
        all_idxs = list(set(all_idxs) - dict_users[i])

    return dict_users

def average_weights(w, fraction):  # this can also be used to average gradients
    """
    :param w: list of weights generated from the users
    :param fraction: list of fraction of data from the users
    :Returns the weighted average of the weights.
    """
    w_avg = copy.deepcopy(w[0]) #copy the weights from the first user in the list 
    for key in w_avg.keys():
        w_avg[key] *= torch.tensor(fraction[0]/sum(fraction), dtype=w_avg[key].dtype)
        for i in range(1, len(w)):
            w_avg[key] += w[i][key] * torch.tensor(fraction[0]/sum(fraction), dtype=w_avg[key].dtype)

    return w_avg

def calculate_gradients(new_weights, old_weights):
    """
    :param new_weights: list of weights generated from the users
    :param old_weights: old weights of a model, probably before training
    :Returns the list of gradients.
    """
    gradients = []
    for i in range(len(new_weights)):
        gradients.append(copy.deepcopy(new_weights[i]))
        for key in gradients[i].keys():
            gradients[i][key] -= old_weights[key]

    return gradients

def update_weights_from_gradients(gradients, old_weights):
    """
    :param gradients: gradients
    :param old_weights: old weights of a model, probably before training
    :Returns the updated weights calculated by: old_weights+gradients.
    """
    updated_weights = copy.deepcopy(old_weights)
    for key in updated_weights.keys():
        updated_weights[key] = old_weights[key] + gradients[key]

    return updated_weights
    


def powersettool(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

def least_core(char_function_dict, N):
    """Solves the least core LP problem.

    Args:
        N: number of participants.
        char_function_dict: dictionary with participants as keys and 
        corresponding characteristic function value as values
    """
    model = pulp.LpProblem('least_core', pulp.LpMinimize)
    x = {i: pulp.LpVariable(name=f'x({i})', lowBound=0) for i in range(1, N+1)}
    e = pulp.LpVariable(name='e')
    model += e # decision variable
    grand_coalition = tuple(i for i in range(1, N+1))
    model += pulp.lpSum(x) == char_function_dict[grand_coalition]
    for key, value in char_function_dict.items():
        model += pulp.lpSum(x[idx] for idx in key) + e >= value
    model.solve()
    print_solution(model)

    return model

def shapley(utility, N):

    shapley_dict = {}
    for i in range(1, N+1):
        shapley_dict[i] = 0
    for key in utility:
        if key != ():
            for contributor in key:
                # print('contributor:', contributor, key) # print check
                marginal_contribution = utility[key] - utility[tuple(i for i in key if i!=contributor)]
                # print('marginal:', marginal_contribution) # print check
                shapley_dict[contributor] += marginal_contribution /((comb(N-1,len(key)-1))*N)

    return shapley_dict

In [ ]:
transform_train = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomCrop(32, padding=4),
    #transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    #transforms.RandomErasing(scale=(0.1, 0.3), ratio=(0.5, 2), value=0)
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = CIFAR10(
    root='./data', train=True, download=True)

test_dataset = CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)

x_train = trainset.data
y_train = np.array(trainset.targets)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
class ResNet9(nn.Module):
    def __init__(self):
        super(ResNet9, self).__init__()
        self.prep = self.convbnrelu(channels=3, filters=64)
        self.layer1 = self.convbnrelu(64, 128)
        self.layer_pool = nn.MaxPool2d(2, 2, 0, 1, ceil_mode=False)
        self.layer1r1 = self.convbnrelu(128, 128)
        self.layer1r2 = self.convbnrelu(128, 128)
        self.layer2 = self.convbnrelu(128, 256)
        self.layer3 = self.convbnrelu(256, 512)
        self.layer3r1 = self.convbnrelu(512, 512)
        self.layer3r2 = self.convbnrelu(512, 512)
        self.out_pool = nn.MaxPool2d(kernel_size=4, stride=4, ceil_mode=False)
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(in_features=512, out_features=10, bias=False)

    def convbnrelu(self, channels, filters):
        layers = []
        layers.append(nn.Conv2d(channels, filters, (3, 3),
                                (1, 1), (1, 1), bias=False))
        layers.append(nn.BatchNorm2d(filters, track_running_stats=False))
        layers.append(nn.ReLU(inplace=True))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.prep(x)
        x = self.layer_pool(self.layer1(x))
        r1 = self.layer1r2(self.layer1r1(x)) 
        x = x + r1
        x = self.layer_pool(self.layer2(x))
        x = self.layer_pool(self.layer3(x))
        r3 = self.layer3r2(self.layer3r1(x))
        x = x + r3
        out = self.out_pool(x)
        out = self.flatten(out)
        out = self.linear(out)
        out = out * 0.125

        return out

class CustomTensorDataset(Dataset):
    """TensorDataset with support of transforms.
    """
    def __init__(self, tensors, transform=None):
        self.tensors = tensors
        self.transform = transform

    def __getitem__(self, index):
        x = self.tensors[0][index]

        if self.transform:
            x = self.transform(x)

        y = self.tensors[1][index]

        return x, y

    def __len__(self):
        return self.tensors[0].shape[0]

In [ ]:
class LocalUpdate(object):

    def __init__(self, lr, local_ep, trainloader):
        self.lr = lr
        self.local_ep = local_ep
        self.trainloader = trainloader

    def update_weights(self, model):

        model.train()
        epoch_loss = []
        optimizer = torch.optim.Adam(model.parameters())
        criterion = nn.CrossEntropyLoss().to(device)
        for iter in range(self.local_ep):
            batch_loss = []
            for batch_idx, (images, labels) in enumerate(self.trainloader):
                images, labels = images.to(device), labels.to(device)
                model.zero_grad()   
                log_probs = model(images)
                loss = criterion(log_probs, labels)
                loss.backward()
                optimizer.step()
                batch_loss.append(loss.item())
            epoch_loss.append(sum(batch_loss)/len(batch_loss))

        return model.state_dict(), sum(epoch_loss) / len(epoch_loss)

def test_inference(model, test_dataset):

    model.eval()
    loss, total, correct = 0.0, 0.0, 0.0
    criterion = nn.CrossEntropyLoss().to(device)
    testloader = DataLoader(test_dataset, batch_size=200, shuffle=False)

    for _, (images, labels) in enumerate(testloader):
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        batch_loss = criterion(outputs, labels)
        loss += batch_loss.item()
        _, pred_labels = torch.max(outputs, 1)
        pred_labels = pred_labels.view(-1)
        correct += torch.sum(torch.eq(pred_labels, labels)).item()
        total += len(labels)
    accuracy = correct / total

    return accuracy, loss

In [ ]:
N = 10 #srch
local_bs = 512
lr = 0.01
local_ep = 5
EPOCHS = 5

noise_rates = np.linspace(0, 1, N, endpoint=False)
split_dset = mnist_iid(trainset, N)
user_groups = {i: 0 for i in range(1, N+1)}
noise_idx = {i: 0 for i in range(1, N+1)}
train_datasets = {i: 0 for i in range(1, N+1)}
for n in range(N):
    user_groups[n+1] = np.array(list(split_dset[n]), dtype=np.int)
    user_train_x, user_train_y = x_train[user_groups[n+1]], y_train[user_groups[n+1]]
    user_noisy_y, noise_idx[n+1] = noisify_MNIST(noise_rates[n], 'symmetric', user_train_x, user_train_y)
    
    train_datasets[n+1] = CustomTensorDataset((user_train_x, user_noisy_y), transform_train)

def fixfuckingbn(subset_weights, global_model_state_dict):
    for pair_1, pair_2 in zip(subset_weights.items(), global_model_state_dict.items()):
        if ('running' in pair_1[0]) or ('batches' in pair_1[0]):
            subset_weights[pair_1[0]] = global_model_state_dict[pair_1[0]]
    
    return subset_weights

global_model = ResNet9().to(device)
global_model.to(device)
global_model.train()

global_weights = global_model.state_dict()
powerset = list(powersettool(range(1, N+1)))

loo_sets = list(i for i in powersettool(range(1, N+1)) if len(i)==N-1)
submodel_dict = {}  
submodel_dict[()] = copy.deepcopy(global_model)
accuracy_dict = {}
loo_array = np.zeros((EPOCHS, N))

In [ ]:
start_time = time.time()

for subset in loo_sets:
    submodel_dict[subset] = copy.deepcopy(global_model)
    submodel_dict[subset].to(device)
    submodel_dict[subset].train()
 
train_loss, train_accuracy = [], []
val_acc_list, net_list = [], []
print_every = 1

idxs_users = np.arange(1, N+1)
total_data = sum(len(user_groups[i]) for i in range(1, N+1))
fraction = [len(user_groups[i])/total_data for i in range(1, N+1)]

for epoch in tqdm(range(EPOCHS)):
    local_weights, local_losses = [], []
    print(f'\n | Global Training Round : {epoch+1} |\n')
    global_model.train()
    for idx in idxs_users:
        trainloader = DataLoader(train_datasets[idx], batch_size=local_bs, shuffle=True)
        local_model = LocalUpdate(lr, local_ep, trainloader)
        w, loss = local_model.update_weights(model=copy.deepcopy(global_model))
        local_weights.append(copy.deepcopy(w))
        local_losses.append(copy.deepcopy(loss))
    global_weights = average_weights(local_weights, fraction) 
    loss_avg = sum(local_losses) / len(local_losses)
    train_loss.append(loss_avg)

    gradients = calculate_gradients(local_weights, global_model.state_dict()) 
    for subset in loo_sets: 
        subset_gradient = average_weights([gradients[i-1] for i in subset], [fraction[i-1] for i in subset])
        subset_weights = update_weights_from_gradients(subset_gradient, submodel_dict[subset].state_dict())
        subset_weights = fixfuckingbn(subset_weights, global_model.state_dict())
        submodel_dict[subset].load_state_dict(subset_weights)

    global_model.load_state_dict(global_weights)
    global_model.eval()

    if (epoch+1) % print_every == 0:
        print(f' \nAvg Training Stats after {epoch+1} global rounds:')
        print(f'Training Loss : {np.mean(np.array(train_loss))}')
        # print('Train Accuracy: {:.2f}% \n'.format(100*train_accuracy[-1]))

    accuracy_dict[powerset[-1]] = test_inference(global_model, test_dataset)[0]

        # Test inference for the sub-models in submodel_dict
    for subset in loo_sets: 
        test_acc, test_loss = test_inference(submodel_dict[subset], test_dataset)
        print(f' \n Results after {epoch} global rounds of training:')
        print("|---- Test Accuracy for {}: {:.2f}%".format(subset, 100*test_acc))
        accuracy_dict[subset] = test_acc
        for i in idxs_users:
            if i not in subset:
                print(i, subset)
                loo_array[epoch, i-1] = accuracy_dict[powerset[-1]] - test_acc

test_acc, test_loss = test_inference(global_model, test_dataset)
print(f' \n Results after {EPOCHS} global rounds of training:')
print("|---- Test Accuracy: {:.2f}%".format(100*test_acc))

accuracy_dict[powerset[-1]] = test_acc


trainTime = time.time() - start_time

print(f'\n ACCURACY: {accuracy_dict[powerset[-1]]}')
print('\n Total Time : {0:0.4f}'.format(trainTime))

  0%|          | 0/5 [00:00<?, ?it/s]


 | Global Training Round : 1 |

 
Avg Training Stats after 1 global rounds:
Training Loss : 2.048456450223923
 
 Results after 0 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 7, 8, 9): 43.58%
10 (1, 2, 3, 4, 5, 6, 7, 8, 9)
 
 Results after 0 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 7, 8, 10): 43.90%
9 (1, 2, 3, 4, 5, 6, 7, 8, 10)
 
 Results after 0 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 7, 9, 10): 43.38%
8 (1, 2, 3, 4, 5, 6, 7, 9, 10)
 
 Results after 0 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 8, 9, 10): 43.68%
7 (1, 2, 3, 4, 5, 6, 8, 9, 10)
 
 Results after 0 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 7, 8, 9, 10): 43.84%
6 (1, 2, 3, 4, 5, 7, 8, 9, 10)
 
 Results after 0 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 6, 7, 8, 9, 10): 42.47%
5 (1, 2, 3, 4, 6, 7, 8, 9, 10)
 
 Results after 0 global rounds of training:
|---- Test Accur

 20%|██        | 1/5 [02:42<10:50, 162.68s/it]

 
 Results after 0 global rounds of training:
|---- Test Accuracy for (2, 3, 4, 5, 6, 7, 8, 9, 10): 41.82%
1 (2, 3, 4, 5, 6, 7, 8, 9, 10)

 | Global Training Round : 2 |

 
Avg Training Stats after 2 global rounds:
Training Loss : 2.013070376992226
 
 Results after 1 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 7, 8, 9): 49.51%
10 (1, 2, 3, 4, 5, 6, 7, 8, 9)
 
 Results after 1 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 7, 8, 10): 49.17%
9 (1, 2, 3, 4, 5, 6, 7, 8, 10)
 
 Results after 1 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 7, 9, 10): 47.55%
8 (1, 2, 3, 4, 5, 6, 7, 9, 10)
 
 Results after 1 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 8, 9, 10): 45.88%
7 (1, 2, 3, 4, 5, 6, 8, 9, 10)
 
 Results after 1 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 7, 8, 9, 10): 47.89%
6 (1, 2, 3, 4, 5, 7, 8, 9, 10)
 
 Results after 1 global rounds of training:
|---- Test Accur

 40%|████      | 2/5 [05:25<08:08, 162.68s/it]

...

 80%|████████  | 4/5 [10:52<02:43, 163.15s/it]

 
 Results after 3 global rounds of training:
|---- Test Accuracy for (2, 3, 4, 5, 6, 7, 8, 9, 10): 56.67%
1 (2, 3, 4, 5, 6, 7, 8, 9, 10)

 | Global Training Round : 5 |

 
Avg Training Stats after 5 global rounds:
Training Loss : 1.8962589448690417
 
 Results after 4 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 7, 8, 9): 64.70%
10 (1, 2, 3, 4, 5, 6, 7, 8, 9)
 
 Results after 4 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 7, 8, 10): 66.03%
9 (1, 2, 3, 4, 5, 6, 7, 8, 10)
 
 Results after 4 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 7, 9, 10): 63.42%
8 (1, 2, 3, 4, 5, 6, 7, 9, 10)
 
 Results after 4 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 6, 8, 9, 10): 62.28%
7 (1, 2, 3, 4, 5, 6, 8, 9, 10)
 
 Results after 4 global rounds of training:
|---- Test Accuracy for (1, 2, 3, 4, 5, 7, 8, 9, 10): 62.28%
6 (1, 2, 3, 4, 5, 7, 8, 9, 10)
 
 Results after 4 global rounds of training:
|---- Test Accu

100%|██████████| 5/5 [13:35<00:00, 163.18s/it]

 
 Results after 4 global rounds of training:
|---- Test Accuracy for (2, 3, 4, 5, 6, 7, 8, 9, 10): 58.69%
1 (2, 3, 4, 5, 6, 7, 8, 9, 10)


 
 Results after 5 global rounds of training:
|---- Test Accuracy: 65.41%

 ACCURACY: 0.6541

 Total Time : 819.2373


In [ ]:
def stats(vector):
    n = len(vector)
    egal = np.array([1/n for i in range(n)])
    normalised = np.array(vector / vector.sum())
    msg = f'Original vector: {vector}\n'
    msg += f'Normalised vector: {normalised}\n'
    msg += f'Max Dif: {normalised.max()-normalised.min()}\n'
    msg += f'Distance: {np.linalg.norm(normalised-egal)}\n'

    msg += f'Budget: {vector.sum()}\n'
    print(msg)

In [ ]:
# STRAIGHT LOO
stats(np.array(loo_array.sum(0)))

Original vector: [ 0.1624  0.1538  0.1354  0.1412  0.1153  0.0278  0.0799  0.0173 -0.0812
 -0.072 ]
Normalised vector: [ 0.23885866  0.22620974  0.19914693  0.2076776   0.16958376  0.04088837
  0.11751728  0.02544492 -0.11942933 -0.10589793]
Max Dif: 0.35828798352698954
Distance: 0.4017210354293261
Budget: 0.6798999999999994



In [ ]:
# WEIGHTED LOO
base = np.arange(1, EPOCHS+1)
stats((np.tile(base,(N,1)).T*loo_array).sum(0))

Original vector: [ 0.6212  0.6055  0.4994  0.5332  0.4194  0.1806  0.3369  0.1128 -0.2296
 -0.182 ]
Normalised vector: [ 0.21439912  0.20898047  0.17236143  0.18402706  0.14475047  0.06233175
  0.11627666  0.03893146 -0.07924346 -0.06281494]
Max Dif: 0.29364257610271305
Distance: 0.3214239296403874
Budget: 2.8973999999999975



In [ ]:
# REPUTATION

stats(np.heaviside(loo_array, 1).mean(0))

Original vector: [1.  1.  1.  1.  1.  0.6 0.8 0.4 0.  0.2]
Normalised vector: [0.14285714 0.14285714 0.14285714 0.14285714 0.14285714 0.08571429
 0.11428571 0.05714286 0.         0.02857143]
Max Dif: 0.14285714285714285
Distance: 0.16288220358559113
Budget: 7.000000000000001

